In [109]:
pip install -U gazpacho

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gazpacho: filename=gazpacho-1.1-py3-none-any.whl size=7483 sha256=628c49bc8aa56da7e05902dcb2cac9d2dc29a3449367acddf529d98ad56b564c
  Stored in directory: /Users/shaungutierrez/Library/Caches/pip/wheels/ec/45/e0/490eb5e25601b4f9425fcde4a0034601c492a29e82268be4d3
Successfully built gazpacho
Note: you may need to restart the kernel to use updated packages.


In [110]:
#Dependencies
import numpy as np
import pandas as pd
import re
import time
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output
from gazpacho import Soup

In [52]:
#import get to call a get request on the site
from requests import get

#get the first page of the east bay housing prices
response = get('https://losangeles.craigslist.org/search/eby/apa?hasPic=1&availabilityMode=0') #get rid of those lame-o's that post a housing option without a pic using their filter

from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')

#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')
print(type(posts)) #to double check that I got a ResultSet
print(len(posts)) #to double check I got 120 (elements/page)

<class 'bs4.element.ResultSet'>
120


In [53]:
#grab the first post
post_one = posts[0]

In [54]:
#grab the price of the first post
post_one_price = post_one.a.text
post_one_price.strip()

'$2,579'

In [4]:
#grab the time of the post in datetime format to save on cleaning efforts
post_one_time = post_one.find('time', class_= 'result-date')
post_one_datetime = post_one_time['datetime']

In [5]:
#title is a and that class, link is grabbing the href attribute of that variable
post_one_title = post_one.find('a', class_='result-title hdrlnk')
post_one_link = post_one_title['href']

#easy to grab the post title by taking the text element of the title variable
post_one_title_text = post_one_title.text

In [6]:
#grabs the whole segment of housing details. We will need missing value handling in the loop as this kind of detail is not common in posts
#the text can be split, and we can use indexing to grab the elements we want. number of bedrooms is the first element.
#sqft is the third element

post_one_num_bedrooms = post_one.find('span', class_ = 'housing').text.split()[0]

post_one_sqft = post_one.find('span', class_ = 'housing').text.split()[2][:-3] #cleans the ft2 at the end

IndexError: list index out of range

In [7]:
#the neighborhood is grabbed by finding the span class 'result-hood' and pulling the text element from that
post_one_hood = posts[0].find('span', class_='result-hood').text

In [ ]:
#build out the loop
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

#each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
pages = np.arange(0, results_total+1, 120)

iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []
lats = []
longs = []

for page in pages:
    
    #get request
    response = get("https://losangeles.craigslist.org/d/apartments-housing-for-rent/search/apa" 
                   + "s=" #the parameter for defining the page number 
                   + str(page) #the page number in the pages array from earlier
                   + "&hasPic=1"
                   + "&availabilityMode=0")

    sleep(randint(1,5))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    #define the html text
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #define the posts
    posts = html_soup.find_all('li', class_= 'result-row')
        
    #extract data item-wise
    for post in posts:

        if post.find('span', class_ = 'result-hood') is not None:

            #posting date
            #grab the datetime element 0 for date and 1 for time
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)

            #post link
            post_link = post_title['href']
            post_links.append(post_link)
                              
            sleep(randint(1,5))

            response = get(post_link)

            soup = BeautifulSoup(response.text, 'html.parser')
            coords = soup.find_all(attrs={"data-latitude": True, "data-longitude": True})

            try:
                lat = coord['data-latitude']
            except:
                lat = None
            lats.append(lat)

            try: 
                long = coord['data-longitude']
            except:
                long = None
            longs.append(long)

            #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            post_price = int(post.a.text.strip().replace("$", "").replace(",", "")) 
            post_prices.append(post_price)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                bedroom_counts.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)

                
    iterations += 1
    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

In [ ]:
import pandas as pd

la_apts = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'number bedrooms': bedroom_counts,
                        'sqft': sqfts,
                        'URL': post_links,
                       'price': post_prices,
                       'Latitude': lats,
                       'Longitude': longs})
print(la_apts.info())
la_apts.head(10)

In [166]:
#first things first, drop duplicate URLs because people are spammy on Craigslist. 
#Let's see how many uniqe posts we really have.
la_apts = la_apts.drop_duplicates(subset='URL')
len(la_apts.drop_duplicates(subset='URL'))

#make the number bedrooms to a float (since np.nan is a float too)
la_apts['number bedrooms'] = la_apts['number bedrooms'].apply(lambda x: float(x))

#convert datetime string into datetime object to be able to work with it
from datetime import datetime

la_apts['posted'] = pd.to_datetime(la_apts['posted'])

#Looking at what neighborhoods there are with la_apts['neighborhood'].unique() allowed me to see what
#I needed to deal with in terms of cleaning those.

#remove the parenthesis from the left and right of the neighborhoods
la_apts['neighborhood'] = la_apts['neighborhood'].map(lambda x: x.lstrip('(').rstrip(')'))

#titlecase them
la_apts['neighborhood'] = la_apts['neighborhood'].str.title()

#just take the first name of the neighborhood list, splitting on the '/' delimiter
la_apts['neighborhood'] = la_apts['neighborhood'].apply(lambda x: x.split('/')[0])

#remove whitespaces
la_apts['neighborhood'] = la_apts['neighborhood'].apply(lambda x: x.strip())

In [178]:
len(la_apts)

120

In [167]:
la_apts.to_csv('Resources/la_apts.csv')

In [168]:
listings = la_apts['URL']

In [169]:
lats = []
longs = []



In [171]:
la_apts = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'number bedrooms': bedroom_counts,
                        'sqft': sqfts,
                        'URL': post_links,
                       'price': post_prices,
                       'Latitude': lats,
                       'Longitude': longs})

ValueError: arrays must all be same length

In [173]:
len(post_timing)

3120

In [174]:
len(post_hoods)

3120

In [175]:
len(bedroom_counts)

3120

In [176]:
len(lats)

120

In [172]:
print(la_apts.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   posted           120 non-null    datetime64[ns]
 1   neighborhood     120 non-null    object        
 2   post title       120 non-null    object        
 3   number bedrooms  103 non-null    float64       
 4   sqft             107 non-null    float64       
 5   URL              120 non-null    object        
 6   price            120 non-null    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 7.5+ KB
None
